## Aziz Saqlain
### M2 Miage - Informatique Décisionnelle

In [ ]:
from pyspark import SparkContext, SparkConf
from time import time

In [ ]:
_5105039_EDGES = "/FileStore/tables/5105039_edges.txt" # 5M edges graph
_905468_EDGES = "/FileStore/tables/905468_edges.txt" # 905K edges graph
_147892_EDGES = "/FileStore/tables/147892_edges.txt" # 147 edges graph

In [ ]:
# Parse the file to get edges
def parseFile(filename):
    lines = sc.textFile(filename)
    lines = lines.filter(lambda x: x[0] != "#") # specific to web_Stanford file, there were lines with this character in the head of the file
    edges = lines.map(lambda x : (x.split("\t")[0],x.split("\t")[1])) # adapt it based on the separation character in the file
    return edges


In [ ]:
# Map Job to form the adjacency list for each node 
def Map(AllEdges): 
  AllEdges = AllEdges.flatMap(lambda x: (x,(x[1],x[0])))
  AdjacencyList = AllEdges.groupByKey().map(lambda x : (x[0], list(x[1])))
  return AdjacencyList

# Reduce job based on the first algorithm
def Reduce(AdjacencyList):
  key, values = AdjacencyList
  valueList = []
  minValue = key
  for value in values:
    if value < minValue:
      minValue = value
    valueList.append(value)
  if minValue < key:
    yield((key, minValue))
    for value in valueList:
      if minValue != value:
        Counter.add(1)
        yield((value, minValue))

# Reduce job based on the second algorithm with secondary sorting
def ReduceSecondarySorting(AdjacencyList):
  key, values = AdjacencyList
  minValue = min(values)
  if minValue < key:
    yield((key, minValue))
    for value in values:
      if minValue != value:
        Counter.add(1)
        yield((value, minValue))

### Running MapReduce without Sorting and with Deduplication on a 5 Million Edges Graph


In [ ]:

# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_5105039_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)

    results = AdjacencyList.flatMap(Reduce).distinct() # With dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)



Iteration n° 1
8546673 new pairs created in 153.8295464515686 seconds

Iteration n° 2
4774134 new pairs created in 141.04539251327515 seconds

Iteration n° 3
3235857 new pairs created in 50.956177949905396 seconds

Iteration n° 4
3852842 new pairs created in 47.914557456970215 seconds

Iteration n° 5
2014247 new pairs created in 30.9709689617157 seconds

Iteration n° 6
94614 new pairs created in 20.923240661621094 seconds

Iteration n° 7
1548 new pairs created in 18.803227186203003 seconds

Iteration n° 8
0 new pairs created in 19.39762544631958 seconds


Total execution time (in seconds) :  483.84073662757874


###Running MapReduce without Deduplication and without Sorting on a 5M Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_5105039_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(Reduce) # Without dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
8546673 new pairs created in 96.18157625198364 seconds

Iteration n° 2
23047041 new pairs created in 152.01387405395508 seconds

Iteration n° 3
38079608 new pairs created in 209.68246364593506 seconds

Iteration n° 4
62337433 new pairs created in 261.6156780719757 seconds

Iteration n° 5
52186653 new pairs created in 318.220401763916 seconds

Iteration n° 6
13668685 new pairs created in 145.61848139762878 seconds

Iteration n° 7
243337 new pairs created in 32.642640352249146 seconds

Iteration n° 8
3112 new pairs created in 14.159132957458496 seconds

Iteration n° 9
0 new pairs created in 14.753023624420166 seconds


Total execution time (in seconds) :  1244.887272119522


### Running MapReduce without Deduplication and with Sorting on a 5M Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_5105039_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(ReduceSecondarySorting) # With sorting, Without dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
8546673 new pairs created in 111.49699020385742 seconds

Iteration n° 2
23047041 new pairs created in 171.50130581855774 seconds

Iteration n° 3
38079608 new pairs created in 206.983393907547 seconds

Iteration n° 4
62337433 new pairs created in 256.6878573894501 seconds

Iteration n° 5
52186653 new pairs created in 304.55614948272705 seconds

Iteration n° 6
13668685 new pairs created in 136.14935398101807 seconds

Iteration n° 7
243337 new pairs created in 29.470798015594482 seconds

Iteration n° 8
3112 new pairs created in 15.739539623260498 seconds

Iteration n° 9
0 new pairs created in 14.989662408828735 seconds


Total execution time (in seconds) :  1247.575050830841


### Running MapReduce with Deduplication and with Sorting on a 5M Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_5105039_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(ReduceSecondarySorting).distinct() # With dedub and sorting

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
8546673 new pairs created in 299.2150287628174 seconds

Iteration n° 2
4774134 new pairs created in 183.68332695960999 seconds

Iteration n° 3
3235857 new pairs created in 77.93373012542725 seconds

Iteration n° 4
3852842 new pairs created in 44.84041237831116 seconds

Iteration n° 5
2014247 new pairs created in 28.46990990638733 seconds

Iteration n° 6
94614 new pairs created in 18.97507071495056 seconds

Iteration n° 7
1548 new pairs created in 18.243247509002686 seconds

Iteration n° 8
0 new pairs created in 17.826489448547363 seconds


Total execution time (in seconds) :  689.1872158050537


In this Section we are going to analyze the results of the four different executions on a graph with 5 million edges, We will compare and analyze them based on several factors:

- Total Execution Time: How long each configuration takes to complete the task. This will help understand the impact of sorting and deduplication on the algorithm's efficiency.

- Number of Iterations: The number of iterations required for the algorithm to converge for each configuration. This can indicate how quickly the algorithm can find the connected components under different conditions.

- Rate of Convergence: How quickly the number of new pairs decreases across iterations in each configuration. A faster decrease suggests a more efficient convergence towards identifying all connected components.

- Impact of Sorting and Deduplication: By comparing the configurations with and without sorting and deduplication, we can assess how these processes affect the performance of the algorithm, both in terms of speed and computational overhead.

- Scalability and Efficiency: The overall scalability and efficiency of the algorithm under different configurations when dealing with a large dataset like a graph with 5 billion edges.

## - 1. No Sorting and With Deduplication

### Iterations and Convergence
- Total Iterations: The algorithm converges in 8 iterations. This consistent iteration count indicates a convergence rate for this graph's structure under the deduplication condition.
- Decrease in New Pairs: Starting with 8,546,673 new pairs in the first iteration and decreasing to 0 by the 8th iteration, the pattern of reduction in new pairs suggests effective progression towards finding the connected components, significantly reducing the number of pairs by the third iteration and tapering off quickly thereafter.


### Execution Time
- Total Execution Time: Approximately 483.84 seconds (~8.06 minutes).
- Iteration Time Variability: The time taken for the first two iterations is notably higher than for subsequent iterations, likely due to the larger volume of data (new pairs) being processed. After the initial iterations, the execution time per iteration decreases significantly, indicating that the majority of work is done early on, with refinement in later iterations.

### Impact of Deduplication
- Efficiency Gain: The deduplication step, while adding some overhead, likely contributes to the overall efficiency by eliminating redundant computations in subsequent iterations. This is evident from the reduction in execution time after the initial iterations.
- Data Processing Overhead: The first two iterations, which are significantly longer than the others, suggest that the overhead of processing and deduplicating a large number of pairs can be higher. However, this initial investment in processing time pays off by reducing the complexity of the task in later iterations.


### Considerations
- Scalability: Handling 5 billion edges and converging in 8 iterations with a total execution time of just over 8 minutes demonstrates good scalability, especially considering the deduplication step's complexity. This suggests that the algorithm can handle large-scale graphs effectively, although optimizations for the initial processing could potentially enhance performance further.
- Comparison with Other Configurations: Comparing these results to executions under different conditions (e.g., with/without sorting, with/without dedup) will highlight the trade-offs between execution time, iteration count, and the computational overhead of additional processing steps like sorting and deduplication.



## - 2. No Sorting and No Deduplication

  - Total Execution Time: 1244.89 seconds
  - Iterations: 9
  - Observations: This configuration exhibits the highest total execution time among the scenarios you've provided. The absence of deduplication leads to a significant increase in the number of iterations and the total computation time. Each iteration generates a large number of new pairs, indicative of considerable redundancy in the processing.

### Impact of Omitting Sorting and Deduplication
The absence of deduplication allows for redundant pairs to be processed repeatedly, significantly increasing the computational workload. Without sorting, the algorithm doesn't benefit from any potential optimizations that could arise from processing data in a sorted order, such as potentially more efficient data locality or merge operations.

### Efficiency and Performance
 This configuration is the least efficient in terms of computational time and resource utilization. The growing number of new pairs in each iteration, especially noticeable in the transition from the first iteration to subsequent ones, highlights the computational cost of processing redundant information. The algorithm takes more time to converge, as evidenced by the higher total execution time and the increased number of iterations.

### Comparison with Other Configurations
When compared to the previous configuration, it's clear that deduplication plays a crucial role in enhancing the algorithm's efficiency. The configurations that include deduplication (with or without sorting) demonstrate significantly reduced total execution times. This comparison underscores the impact of redundancy on the computational efficiency of MapReduce algorithms, especially in the context of large-scale graph processing.

The "No Sorting and No Deduplication" scenario effectively serves as a baseline that illustrates the inherent inefficiencies of processing large-scale graph data without mechanisms to eliminate redundancy. The significant increase in total execution time and iterations compared to scenarios with deduplication emphasizes the critical role of deduplication in optimizing performance. While sorting may have mixed effects based on its computational overhead and the specific characteristics of the data being processed, deduplication unequivocally contributes to reducing unnecessary computations, thereby enhancing the overall efficiency of the MapReduce algorithm.


## - 3. Sorting and No Deduplication

- Total Execution Time: 1247.58 seconds
- Iterations: 9
- Observations: The inclusion of sorting without deduplication does not significantly affect the total execution time or the number of iterations compared to when sorting is not used. This indicates that sorting, in the absence of deduplication, does not contribute - significantly to reducing computational redundancy.

### Impact of Deduplication
 Deduplication has a dramatic impact on both the total execution time and the number of iterations required for convergence. It effectively reduces redundancy, leading to fewer computations and a faster convergence. The comparison between runs with and without deduplication (483.84 seconds vs. 1244.89 seconds and 1247.58 seconds) highlights its importance.

### Impact of Sorting
 The sorting operation, when used without deduplication, does not significantly impact the performance. This suggests that the primary factor contributing to efficiency is not the order of processing but the elimination of redundant pairs. However, it's worth noting that sorting might have a more pronounced effect in conjunction with deduplication, as it could potentially further optimize the processing order for deduplication, although such a configuration was not explicitly reported here.

### Iterations and Convergence
 The algorithm consistently requires more iterations when deduplication is not used, which directly correlates with the increased execution time. The pattern of new pairs created in each iteration, especially the exponential growth in the case without deduplication, indicates the compounding effect of redundant computations on the algorithm's efficiency.

 Deduplication emerges as a critical factor in optimizing the performance of the MapReduce algorithm for finding connected components in large graphs. It significantly reduces both the execution time and the number of iterations needed for convergence by eliminating redundant computations. Sorting, in the absence of deduplication, does not markedly improve performance, suggesting that its potential benefits are overshadowed by the costs of processing redundant data. For large-scale graph processing tasks, focusing on reducing redundancy through techniques like deduplication should be a priority to enhance computational efficiency.

## - 4. With Sorting and Deduplication
- Total Execution Time: 689.19 seconds
- Iterations: 8
- Observations: This configuration shows a balance between the number of iterations and total execution time. The presence of both sorting and deduplication results in a higher total execution time compared to without sorting but with deduplication (483.84 seconds), yet significantly lower than the scenarios without deduplication (1244.89 seconds and 1247.58 seconds). The number of new pairs created decreases consistently, indicating effective convergence.

### Impact of Both Sorting and Deduplication
When sorting is combined with deduplication, there's an increase in execution time compared to deduplication alone. This suggests that while deduplication significantly reduces redundancy and hence computation time, the addition of sorting introduces overhead that, despite potentially optimizing processing order, increases the total computation time due to the cost of sorting operations.

### Efficiency and Performance
The configuration with both sorting and deduplication strikes a middle ground in terms of efficiency and performance. It ensures the elimination of redundant computations through deduplication and potentially optimizes the merge phase with sorting, albeit at the cost of added execution time due to sorting overhead.

### Comparative Advantage
Compared to configurations without deduplication, the significant reduction in total execution time underscores the importance of deduplication in managing computational redundancy. Sorting, while beneficial in organizing data, appears to add a layer of complexity that does not necessarily translate into proportional performance gains in this context.

The analysis across different configurations highlights the critical role of deduplication in optimizing MapReduce algorithms for processing large graphs. Deduplication directly impacts the algorithm's efficiency by reducing unnecessary computations. Sorting, on the other hand, while theoretically beneficial for organizing data for faster access and processing, incurs a performance penalty that may outweigh its benefits, especially when combined with deduplication.

This exploration suggests that for large-scale graph processing tasks where performance and computational efficiency are paramount, deduplication should be prioritized. Sorting may be considered based on the specific requirements of the task and the computational resources available, keeping in mind the potential for increased execution times.

### Running MapReduce with Deduplication and without Sorting on a 905K Edges Graphe 

In [ ]:

# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_905468_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)

    results = AdjacencyList.flatMap(Reduce).distinct() # With dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)



Iteration n° 1
1601674 new pairs created in 26.270533561706543 seconds

Iteration n° 2
1385794 new pairs created in 12.590238332748413 seconds

Iteration n° 3
363772 new pairs created in 5.263671159744263 seconds

Iteration n° 4
16826 new pairs created in 2.795743465423584 seconds

Iteration n° 5
0 new pairs created in 2.1762216091156006 seconds


Total execution time (in seconds) :  49.0964081287384


### Running MapReduce without Deduplication and with Sorting on a 905K Edges Graphe 

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_905468_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(ReduceSecondarySorting) # With sorting, Without dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
1601674 new pairs created in 10.679173946380615 seconds

Iteration n° 2
4716470 new pairs created in 12.206235647201538 seconds

Iteration n° 3
4729314 new pairs created in 14.576035737991333 seconds

Iteration n° 4
1649906 new pairs created in 7.780094861984253 seconds

Iteration n° 5
35388 new pairs created in 2.069979190826416 seconds

Iteration n° 6
0 new pairs created in 1.3901245594024658 seconds


Total execution time (in seconds) :  48.70164394378662


### Running MapReduce without Deduplication and without Sorting on a 905K Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_905468_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(Reduce) # Without dedub, without sorting

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
1601674 new pairs created in 10.079532146453857 seconds

Iteration n° 2
4716470 new pairs created in 11.573772430419922 seconds

Iteration n° 3
4729314 new pairs created in 14.936846017837524 seconds

Iteration n° 4
1649906 new pairs created in 8.23173975944519 seconds

Iteration n° 5
35388 new pairs created in 2.2197372913360596 seconds

Iteration n° 6
0 new pairs created in 1.472999095916748 seconds


Total execution time (in seconds) :  48.5146267414093


### Running MapReduce with Deduplication and with Sorting on a 905K Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_905468_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(ReduceSecondarySorting).distinct() # With dedub and sorting

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
1601674 new pairs created in 15.838926553726196 seconds

Iteration n° 2
1385794 new pairs created in 10.067445993423462 seconds

Iteration n° 3
363772 new pairs created in 4.012532711029053 seconds

Iteration n° 4
16826 new pairs created in 2.031034231185913 seconds

Iteration n° 5
0 new pairs created in 1.849839687347412 seconds


Total execution time (in seconds) :  33.799779176712036


In this Section we are going ti analyze the results of the four different executions on a graph with 905K edges, We will compare and analyze them based on sethe factors we've seen in the previous graph.

##- 1. No Sorting and With Deduplication
### Iterations and Execution Time
- Total Iterations: 5
- Total Execution Time: Approximately 49.1 seconds

### Efficiency
The algorithm efficiently reduces the number of pairs across iterations, rapidly converging to 0 new pairs by the 5th iteration. This indicates that deduplication significantly helps in minimizing redundant computations and quickly identifying unique connections within the graph, thus speeding up the convergence of finding connected components.

### Execution Time
The total execution time is relatively short, suggesting that for graphs of this size (905K edges), the computational overhead introduced by deduplication does not significantly impact performance. In fact, it aids in reducing total execution time by avoiding unnecessary iterations over duplicate pairs.

### Iteration-wise Performance
The first iteration, naturally, takes the longest time as it has to process the entire edge set. However, the execution time drops sharply in subsequent iterations. This drop is due to both the reduction in the number of pairs to be processed and the efficiency gains from not having to handle duplicates.

### Impact of Deduplication
Deduplication appears to have a positive effect on both the speed of convergence and the overall execution time. By removing duplicates, the algorithm can focus on processing only meaningful pair relationships, which likely contributes to the quick reduction in new pairs created after each iteration.

### Comparison to Larger Graph
While it's expected that the smaller graph would take less time to process than a larger one, the efficiency gains from deduplication might be more pronounced in larger datasets with a higher potential for duplicate pairs. The relatively linear decrease in processing time from iteration to iteration suggests good scalability for larger graphs.

## - 2. No Sorting and No Deduplication
### Iterations and Execution Time
Total Iterations: 6
Total Execution Time: Approximately 48.5 seconds

### Overall Efficiency
Despite the absence of both deduplication and sorting, the algorithm completes in a comparable total execution time to the configurations with sorting and/or deduplication. This result is somewhat surprising because both sorting and deduplication are generally expected to improve performance by reducing computational redundancy and optimizing data processing, respectively.

### Impact of Omitting Deduplication and Sorting
Not employing deduplication means the algorithm processes every pair generated, including duplicates. The expectation would be that this significantly increases the computational load. However, the execution time remains competitive, suggesting that the Spark framework's handling of these operations is highly efficient, or that the overhead of deduplication and sorting might not be justified for graphs of this size.

### Comparison with Deduplication and Sorting
The total execution time being nearly on par with scenarios employing deduplication and/or sorting is noteworthy. It suggests that for smaller graphs (like the 905K edges graph in this scenario), the benefits of deduplication and sorting might be less pronounced than expected. This could be due to the overhead these processes introduce or the inherent efficiency of the MapReduce framework in managing and processing the data.

### Efficiency and Scalability
This configuration's efficiency, indicated by the relatively low total execution time and the small number of iterations to convergence, raises interesting questions about the scalability of these findings. For larger graphs, the absence of deduplication, in particular, could lead to exponentially larger computational workloads, potentially making the inclusion of deduplication and sorting more critical for maintaining performance.

The performance of the No Dedup and No Sorting configuration on a 905K edges graph illustrates the complexity of optimizing graph processing algorithms. While conventional wisdom suggests that deduplication and sorting should improve efficiency, in smaller graphs, the benefits may not be as significant, possibly due to the overhead these processes introduce. This analysis highlights the importance of context in algorithm optimization—what works for one graph size or topology might not be as effective for another. For developers and researchers, this underscores the need for empirical testing and performance profiling in their specific use cases to determine the most effective optimization strategies.

## - 3. Sorting and No Deduplication

### Iterations and Execution Time
- Total Iterations: 6
- Total Execution Time: Approximately 48.7 seconds

### Efficiency of Sorting Without Deduplication
This configuration demonstrates that sorting alone can significantly improve the performance of graph processing tasks, even without deduplication. The sorting likely aids in more efficient data access and processing, leading to quicker iterations.

### Performance Insights
Despite not employing deduplication, this configuration still performs quite efficiently, with a total execution time close to that observed in scenarios with deduplication. This highlights the impact of sorting on enhancing algorithm efficiency by itself.

### Comparison with Other Configurations
When compared to the scenario with both deduplication and sorting, this configuration takes slightly more time to complete. This suggests that while sorting contributes significantly to performance improvement, deduplication adds an additional layer of efficiency by reducing the volume of data that needs to be processed.

### Scalability and Practical Implications
The results suggest that for medium-sized graphs, like the 905K edges graph, sorting alone can offer substantial performance improvements. However, for even larger datasets or more complex graph structures, incorporating deduplication could provide further benefits. This configuration's efficiency underscores the importance of sorting in processing workflows, especially in scenarios where deduplication might not be feasible due to data characteristics or other constraints.


The  Sorting and No Deduplication configuration on a 905K edges graph illustrates the significant role sorting plays in enhancing the performance of graph processing tasks. While the total execution time is slightly higher than in scenarios employing both deduplication and sorting, it still marks a considerable improvement over configurations without any optimization. This analysis suggests that sorting can be a critical optimization technique in graph processing, especially when combined with other strategies like deduplication for maximizing efficiency. Developers and researchers should consider sorting as a valuable tool in their optimization toolkit, particularly for applications where data integrity and processing speed are paramount.

## - 4. With Sorting and Deduplication

### Iterations and Execution Time
- Total Iterations: 5
- Total Execution Time: Approximately 33.8 seconds

### Efficiency of Deduplication and Sorting
This configuration shows a notable improvement in efficiency, with the total execution time being significantly lower than the other configurations. The use of deduplication likely reduced the number of pairs processed in each iteration by eliminating duplicates, and sorting might have optimized the algorithm's ability to quickly identify and eliminate potential pairs, leading to faster iterations.

### Impact on Performance
The combination of deduplication and sorting leads to a more streamlined process where unnecessary computations are minimized. This is evident in the reduced number of iterations needed to converge to a solution and the overall shorter execution time compared to scenarios without these optimizations.

### Comparison with Other Configurations
This configuration outperforms the others in terms of total execution time, highlighting the importance of deduplication and sorting in optimizing graph processing tasks. It demonstrates that for graphs of this size, employing both deduplication and sorting can lead to substantial performance gains.

### Scalability and Applicability
The significant improvement in execution time suggests that for graphs of similar size to the 905K edges graph, employing both deduplication and sorting is highly beneficial. This might also imply scalability benefits for larger graphs, where the computational overhead can grow exponentially. However, the balance between the overhead introduced by these processes and their efficiency gains would need to be evaluated for larger datasets.

The performance of the with Deduplication and Sorting configuration on a 905K edges graph illustrates the substantial benefits of employing both deduplication and sorting in graph processing tasks. This configuration not only reduced the total execution time but also decreased the number of iterations needed, indicating a more efficient processing pathway. For developers and researchers working with graph data, this suggests that incorporating deduplication and sorting, especially for tasks involving large numbers of edges and nodes, can lead to significant improvements in performance. However, the trade-offs between the overhead of these processes and their benefits should be carefully considered, particularly as the size and complexity of the graph data increase.

### Running MapReduce with Deduplication and without Sorting on a 147K Edges Graph

In [ ]:

# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_147892_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)

    results = AdjacencyList.flatMap(Reduce).distinct() # With dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
222478 new pairs created in 5.032658100128174 seconds

Iteration n° 2
482052 new pairs created in 6.997102975845337 seconds

Iteration n° 3
903885 new pairs created in 7.676016092300415 seconds

Iteration n° 4
486013 new pairs created in 4.058413982391357 seconds

Iteration n° 5
10026 new pairs created in 1.6132934093475342 seconds

Iteration n° 6
0 new pairs created in 1.7075254917144775 seconds


Total execution time (in seconds) :  27.085010051727295


### Running MapReduce without Deduplication and with Sorting on a 147K Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_147892_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(ReduceSecondarySorting) # With sorting, Without dedub

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
222478 new pairs created in 2.734717845916748 seconds

Iteration n° 2
713285 new pairs created in 3.0803163051605225 seconds

Iteration n° 3
1517297 new pairs created in 4.395355939865112 seconds

Iteration n° 4
2232324 new pairs created in 6.379872798919678 seconds

Iteration n° 5
1231906 new pairs created in 5.422848463058472 seconds

Iteration n° 6
26072 new pairs created in 1.5698208808898926 seconds

Iteration n° 7
0 new pairs created in 1.2431821823120117 seconds


Total execution time (in seconds) :  24.826114416122437


### Running MapReduce without Deduplication and without Sorting on a 147K Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_147892_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(Reduce) # Without dedub, without sorting

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
222478 new pairs created in 2.9519357681274414 seconds

Iteration n° 2
713285 new pairs created in 3.005495309829712 seconds

Iteration n° 3
1517297 new pairs created in 4.452477931976318 seconds

Iteration n° 4
2232324 new pairs created in 7.004806041717529 seconds

Iteration n° 5
1231906 new pairs created in 5.945910692214966 seconds

Iteration n° 6
26072 new pairs created in 1.7517974376678467 seconds

Iteration n° 7
0 new pairs created in 1.1598916053771973 seconds


Total execution time (in seconds) :  26.27231478691101


### Running MapReduce with Deduplication and with Sorting on a 147K Edges Graph

In [ ]:
# Global variables
iteration = 0
Counter = sc.accumulator(1) # A global accumulator to start the loop
executionTime = 0
results = [] # Output of the Reduce phase
edges = parseFile(_147892_EDGES)

# We stop the loop when we can't form a newPair 
while (Counter.value != 0):
    # Start a timer for each iteration
    beginTime = time()
    iteration += 1
    print("\nIteration n°", iteration)

    # Set back the accumulator to 0 for the Reduce Job to increment it if a newPair is formed
    Counter = sc.accumulator(0)

    # Map Job
    AdjacencyList = Map(edges)
    
    results = AdjacencyList.flatMap(ReduceSecondarySorting).distinct() # With dedub and sorting

    results.collect()
    edges = results # To ensure the results (which are a RDD) are updated after each iteration, we store it in the global variable edges 

    endTime = time()  
    print(Counter.value, "new pairs created in", endTime - beginTime, "seconds")
    executionTime += (endTime - beginTime)

print("\n\nTotal execution time (in seconds) : ", executionTime)


Iteration n° 1
222478 new pairs created in 5.10714864730835 seconds

Iteration n° 2
482052 new pairs created in 6.978255987167358 seconds

Iteration n° 3
903885 new pairs created in 7.94918417930603 seconds

Iteration n° 4
486013 new pairs created in 4.1683502197265625 seconds

Iteration n° 5
10026 new pairs created in 1.682548999786377 seconds

Iteration n° 6
0 new pairs created in 1.4777169227600098 seconds


Total execution time (in seconds) :  27.363204956054688


In this Section we are going ti analyze the results of the four different executions on a graph with 147K edges, We will compare and analyze them based on sethe factors we've seen in the previous graph.

##- 1. No Sorting and With Deduplication

### Iterations and Execution Time
- Total Iterations: 6
- Total Execution Time: Approximately 27.08 seconds

### Effectiveness of Deduplication Without Sorting
This configuration demonstrates that deduplication significantly enhances the efficiency of graph processing tasks, even in the absence of sorting. Deduplication ensures that each unique pair is processed only once, reducing the computational load and streamlining the expansion of pairs.

### Performance Insights
The total execution time of approximately 27.08 seconds underscores the impact of deduplication on improving processing speed. By eliminating duplicate pairs early in the processing pipeline, the algorithm avoids unnecessary work, leading to faster overall execution.

### Scalability and Practical Implications
The results suggest that for smaller to medium-sized graphs, such as the 147K edges graph, deduplication without sorting can offer significant performance improvements. This configuration is particularly beneficial in scenarios where sorting is not feasible or where the primary goal is to minimize redundant data processing.

this configuration a 147K edges graph highlights the crucial role deduplication plays in enhancing the performance of graph processing tasks. The substantial reduction in execution time, compared to scenarios without optimization, illustrates deduplication's effectiveness in streamlining data processing. This analysis suggests that deduplication is a powerful optimization technique, particularly valuable in scenarios where maintaining data integrity and reducing computational overhead are paramount. Developers and researchers might find deduplication especially useful in applications where the primary challenge is managing the volume of data rather than the complexity of data relationships.

##- 2. With Sorting and No Deduplication

### Iterations and Execution Time:
- Total Iterations: 7
- Total Execution Time: Approximately 24.83 seconds


### Efficiency of Sorting Without Deduplication
This setup illustrates that sorting significantly improves the efficiency of graph processing tasks, even in the absence of deduplication. Sorting aids in organizing the data in a manner that likely accelerates the identification and processing of new pairs, contributing to a more streamlined execution.

### Performance Insights
The total execution time of around 24.83 seconds, the shortest among the configurations discussed, underscores the substantial impact of sorting on processing speed. By optimizing the order in which elements are processed, sorting minimizes computational overhead, leading to faster iterations and overall quicker convergence.

### Comparison with Other Configurations
 When compared to the scenario with deduplication (and without sorting), this configuration showcases a different optimization strategy. While deduplication directly reduces the volume of data to be processed by removing duplicates, sorting optimizes the processing flow itself, enhancing efficiency from a different angle.

### Scalability and Practical Implications
The results suggest that for graphs of this scale (147K edges), sorting alone can offer significant performance improvements, making it a compelling choice in scenarios where deduplication might not be possible or necessary. This approach is especially relevant in cases where the primary optimization goal is to enhance the processing flow rather than to reduce data volume.


The "With Sorting and No Deduplication" configuration on a 147K edges graph demonstrates the critical role of sorting in enhancing graph processing tasks' performance. This setup achieved the shortest total execution time among the various configurations, highlighting sorting as a powerful optimization technique. The analysis underscores sorting's potential to significantly improve algorithm efficiency, particularly in scenarios where the aim is to optimize the processing sequence rather than to eliminate data redundancy. Developers and researchers should consider sorting as an essential tool in their optimization toolkit for graph processing tasks, especially when deduplication is not a viable option.

## - 3. No Sorting and No Deduplication

### Iterations and Execution Time
- Total Iterations: 7
- Total Execution Time: Approximately 26.27 seconds

### Impact of Absence of Sorting and Deduplication
This configuration demonstrates the baseline performance of graph processing tasks without the implementation of any optimization techniques. The absence of sorting and deduplication means that the data is processed in its original order, potentially leading to less efficient data access and processing patterns.

### Comparison with Other Configurations
Compared to the configuration with sorting and no deduplication, this scenario reveals the inherent efficiency of sorting as an optimization strategy. While sorting alone has shown to significantly reduce execution time, the absence of both sorting and deduplication only marginally increases the total execution time, highlighting the effectiveness of sorting over deduplication in this particular context.

The configuration without sorting and without deduplication on a 147K edges graph offers insight into the baseline performance of graph processing tasks. While the total execution time is slightly longer than that observed with sorting, the difference is not substantial, suggesting that the impact of optimization techniques may vary based on the specific characteristics of the graph. This analysis underscores the importance of a tailored approach to optimization, where the choice of techniques such as sorting and deduplication should be informed by the specific goals and constraints of the processing task.

## - 4. With Sorting and Deduplication

### Iterations and Execution Time
- Total Iterations: 6
- Total Execution Time: Approximately 27.36 seconds

### Efficiency of Sorting and Deduplication
This configuration demonstrates the combined effectiveness of sorting and deduplication in optimizing the performance of graph processing tasks. Sorting aids in more efficient data access and processing by organizing data in a manner that potentially reduces the computational complexity of finding new pairs. Deduplication further enhances efficiency by eliminating redundant computations, thereby reducing the total number of operations required.

### Performance
The total execution time of around 27.36 seconds, while comparable to the scenario without sorting but with deduplication, underscores the contribution of both sorting and deduplication towards optimizing the algorithm's performance. It highlights that while each technique independently contributes to performance improvement, their combined application can achieve a balanced optimization effect.

### Comparison with Other Configurations
When compared to configurations without these optimizations or with only one of them applied, this configuration demonstrates a balanced approach to efficiency, leveraging both sorting for data organization and deduplication for reducing computational load. This balanced optimization might not be as fast as sorting alone in some contexts but provides a comprehensive approach to handling large datasets.

The configuration with both sorting and deduplication on a 147K edges graph demonstrates the significant role these optimizations play in enhancing the efficiency of graph processing tasks. The total execution time and the pattern of iterations indicate a methodical reduction in the computational complexity of the task, leading to a quicker and more efficient processing outcome. This analysis reinforces the value of combining multiple optimization techniques to address the challenges of processing large and complex graphs, suggesting that developers and researchers should consider both sorting and deduplication as integral components of their optimization toolkit for graph processing applications.